In [ ]:
# 初始化 Word2Vec：

if ('init' == phase):
    # 定义模型保存位置
    paths['dir.modelroot'] = os.path.join(paths['dir.dataroot'], '..', 'models')
    for modeltool in ['gensim', 'TensorFlow']:
        for embedsource in ['corpus']:
            dname = os.path.join(paths['dir.modelroot'], '{}.{}'.format(modeltool, embedsource))
            if not os.path.isdir(dname):
                os.mkdir(dname)
            paths['dir.{}.{}'.format(modeltool, embedsource)] = dname

In [ ]:
# 建立【词】表示

if 'modeling.word' == phase:
    # Collect sentences from raw data into LineSentence
    sentences = {}
    pathtmp = {}
    pathtmp['root'] = os.path.join(paths['dir.dataroot'], 'trialdata')
    for tpart in ['train']: #, 'test']:
        pathtmp[tpart] = os.path.join(pathtmp['root'], tpart)
        sentences[tpart] = []
        folderList = os.listdir(pathtmp[tpart])
        for folder in folderList:
            fileList = os.listdir(os.path.join(pathtmp[tpart], folder))
            for eachf in fileList:
                fpathtmp = os.path.join(pathtmp[tpart], folder, eachf)
                with open(fpathtmp, 'r') as f:
                    sentences[tpart].append(f.read())
        #save sentences in file
        sentencePath = os.path.join(pathtmp['root'], 'sentences-{}'.format(tpart))
        with open(sentencePath, 'w') as f:
            for sentence in sentences[tpart]:
                f.write(sentence)
                f.write('\n')
                
    sentences = gensim.models.word2vec.LineSentence(sentencePath)
                
    # Create model of word
    model = gensim.models.word2vec.Word2Vec(iter=15, size=vecsize, seed=19, sg=1, window=30, workers=12, negative=3)#hs=1)#
#     print("Model created.")
    
    # Set trimmer
    if 'NO' == idpart['useStopwords']:
        trimmer = lambda word, count, min_count: gensim.utils.RULE_KEEP
    else:
        import nltk
        
        wordset = []        
        for tpart in ['dir.train']:
            dirpath = paths[tpart]
            for cls in os.listdir(dirpath):
                clspath = os.path.join(dirpath, cls)
                files = os.listdir(clspath)
                for f in files:
                    fpath = os.path.join(clspath, f)
                    with open(fpath, 'r', encoding="latin-1") as fpath_f:
                        tmpFileStr = fpath_f.read()
                    tmpFileStr = tmpFileStr.split()
                    wordset += tmpFileStr

        wordDist = nltk.FreqDist(wordset)
        freqword = [i[0]for i in wordDist.most_common(30)]
        stopwords = freqword
        
        trimmer = lambda word, count, min_count: gensim.utils.RULE_DISCARD if word in stopwords else gensim.utils.RULE_KEEP
#     print('Trimmer set.')
    
    # Build vocabulary
    model.build_vocab(sentences=sentences, trim_rule=trimmer)
#     print("Vocabulary creation finished.")
    
    # Train model of word
    model.train(sentences=sentences, total_examples=model.corpus_count, epochs=model.iter)
#     print('Training finished.')
    
    # save model
    calander = datetime.date.today().timetuple()
    modelpath = os.path.join(
        paths['dir.{}.{}'.format(idpart['tool.word'], 'corpus')],
        '{}.{}{}{}'.format(idpart['tool.word'], calander.tm_year, calander.tm_mon, calander.tm_mday)
    )
    model.save(modelpath)
#     print("save model finished")

In [ ]:
if ('modeling.document' == phase):
    w2v_tag = 'w2v.ArithmeticMean'
    
    # Read data and save it in data['vec.train'] 和 data['vec.test']
    data = {}
    data['vec.train'] = {w2v_tag:[], 'class':[]}
    data['vec.test'] = {w2v_tag:[], 'class':[]}
    
    for tpart in ['train', 'test']:
        dirpath = paths['dir.{}'.format(tpart)]
        for (ind, cls) in enumerate(os.listdir(dirpath)):
            clspath = os.path.join(dirpath, cls)
            files = os.listdir(clspath)
            for f in files:
                fpath = os.path.join(clspath, f)
                # Start to represent a document
                with open(fpath, 'r') as readf:
                    tokens = [token for token in readf.read().split()]
#                     if 'ArithmeticMean' == idpart['model.document']:
                        # Word2Vec representation: Arithmetic Mean
                        # begin
                    vec = np.array([0.0 for i in range(vecsize)])
                    expectationVal = np.array([0.0 for i in range(vecsize)])
                    countvec = 0
                    for token in tokens:
                        try:
                            vec += model[token]
                            countvec += 1
                        except:
                            vec += expectationVal
                    vec = vec / float(countvec)
                data['vec.{}'.format(tpart)][w2v_tag].append(vec)
                data['vec.{}'.format(tpart)]['class'].append(cls)
    
    # Save in Pandas.DataFrame
    # 将 data['matrix.train'] 与 data['matrix.test'] 转换成 Pandas.DataFrame 格式，保存到 df['train'] 和 df['test'] 中（df 为字典格式：String -> DataFrame）
    df = {}

    for tpart in ['train', 'test']:
        datadict = {}
        datadict['class'] = data['vec.{}'.format(tpart)]['class']
        datavec = np.array(data['vec.{}'.format(tpart)][w2v_tag])
        for col in range(vecsize):#+1): #
            datadict[col]= datavec[:, col]
    
        df[tpart] = pd.DataFrame(data=datadict)
        df[tpart] = df[tpart].sample(frac=1)
        df[tpart].reset_index(drop=True, inplace=True)
        display(df[tpart])
    
    # 繁琐点：研究如何把 CSR 矩阵中的数据规整好放到 DataFrame 中，并与 Class 一一对应
            
    # Store data in X_train, y_train, X_test, y_test
    ##train
    X_train, y_train = df['train'].drop('class', axis=1), df['train']['class']
    ##test
    X_test, y_test_true = df['test'].drop('class', axis=1), df['test']['class']